In [ ]:
import numpy as np
from typing import Callable, Any


class IncorrectArchitectureError(Exception):
        pass

class IncorrectDatabaseFileFormatError(Exception):
        pass

class IncorrectNumberOfParametersError(Exception):
        pass

def ReLu(value):
        return np.maximum(0, value)


class neural_network:
        """
        neural_network CLASS

        Parameters:
                architecture: An array containing number of neurons in each layer of created Neural Network
                database_file: NPZ type file, which contains data about weights and biases of created Neural Network
        
        Description:
                Purpose of this class is to facilitate creating Neural Network object. 
                User needs to define an array which describes specific architecture for created Neural Network.
                This architercures`s ought to contain number of perceptrons in each layer.
                Length of the array is interpreted as number of layers, which newly created Neural Network will contain.
                Database file, which is needed for storing data about weights and biases, needs to be provided by user.
        """
        def __init__(self, architecture: list[int], database_file: str) -> None:
                # Checking for error connected to providing only one layer
                if len(architecture) <= 1:
                        raise IncorrectArchitectureError("Your neural network must consists of at least 2 layers")
                # This Neural Network builder only manage databases in form of NPZ files
                # Providing other others raise and error
                if not database_file.endswith(".npz"):
                        raise IncorrectDatabaseFileFormatError("Model of this neural network only works on NPZ files")

                self._architecture: list[int] = architecture
                self._database_file: str = database_file
                
                # Creating empty arrays for paramaters for Neural Network
                self._weights_matrices: list[np.ndarray]  = []
                self._biases_matrices: list[np.ndarray] = []
                self._activation_functons: list[Callable[[np.ndarray], np.ndarray]] = []

                number_of_layers: int = len(architecture)
                for layer_index in range(number_of_layers - 1):
                        # Initializing weights of Neural Network with random numbers from range 0 to 1
                        layer_weights_matrix: np.ndarray = np.random.rand(self._architecture[layer_index], self._architecture[layer_index + 1])
                        # Initializing biases of Neural Network with zeros
                        layer_biases_matrix: np.ndarray = np.zeros((1, self._architecture[layer_index + 1]))

                        # Adding newly created matrixes to special arrays
                        self._weights_matrices.append(layer_weights_matrix)
                        self._biases_matrices.append(layer_biases_matrix)


        """
        GETTERS METHODS
        """
        @property
        def architecture(self) -> list[np.ndarray]:
                """
                Getter of architecture array, which describes structure of Neural Network
                """
                return self._architecture
        @property
        def weights_matrixes(self) -> list[np.ndarray]:
                """
                Getter for array, which contains weights`s matrixes of each layer in Neural Network
                """
                return self._weights_matrices
        @property
        def biases_matrixes(self) -> list[np.ndarray]:
                """
                Getter for array, which contains biases`s matries of each layer in Neural Network
                """
                return self._biases_matrices
        @property
        def activation_functions(self) -> list[Callable[[np.ndarray], np.ndarray]]:
                """
                Getter for array, which contains activation function for neurons in each layer in Neural Network
                """
                return self._activation_functons

        
        def assign_activation_functions(self, activation_functions: list[Callable[[np.ndarray], np.ndarray]]) -> None:
                """
                Setter method for changing user-provided activation functions

                Parameters:
                        activation_functions: List of callable functions for each Neural Network`s layer
                        Warning: Order of activation functions must correspond to which layer each function is assigned
                """
                # Checking for an error connected to providing incorrect number of activation functions
                if len(activation_functions) != (len(self._architecture) - 1):
                        raise IncorrectNumberOfParametersError("Number of provided activation functions is incorrect")
                
                # Assiging new activation functions for Neural Network
                self._activation_functons = activation_functions

                return None

        
        def front_propagation(self, input_matrix: np.ndarray, activation_functions: list[Callable[[np.ndarray], np.ndarray]]) -> tuple[np.ndarray, dict[str, Any]]:
                """
                front_propagation METHOD

                Parameters:
                        input_matrix: Matrix which contains input values for Neural Network
                        activation_functions: List of callable functions for each Neural Network`s layer
                        Warning: Order of activation functions must correspond to which layer each function is assigned
                
                Returns:
                        current_activations, cache: Tuple of numpy array and dictionary, which conatins pre and post activation values for sums in each layer
                
                """
                # Checking if number of activation functions match number of layers in Neural Network
                if len(activation_functions) != (len(self._architecture) - 1):
                        raise IncorrectNumberOfParametersError("Number of activation function is incorrect!")
                # Checking if provided input matrix has proper sizes
                if input_matrix.shape[0] != self._architecture[0]:
                        raise IncorrectNumberOfParametersError("Size of input matrix is incorrect!")
                
                current_activations: np.ndarray = input_matrix
                # Creating cache with pre and post activation values of sums on each neuron
                # This results will be used later in back propagation algorithm 
                cache: dict[str, Any] = {'A': [input_matrix]}
                cache['Z'] = []

                # Itterating through Neural Network to achieve correct post actiovation sums on output perceptrons
                for layer_index in range(len(self._architecture) - 1):
                        # Assigning weights, biases and activation functions to new variable for better readability
                        current_weights: np.ndarray = self._weights_matrices[layer_index]
                        current_biases: np.ndarray = self._biases_matrices[layer_index]
                        current_activation_function: Callable[[np.ndarray], np.ndarray] = activation_functions[layer_index]

                        # Calculating pre activation values for sums on each neuron in specific layer with adding proper biases to it
                        pre_activation: np.ndarray = current_activations @ current_weights + current_biases
                        # Saving result to cache dictionary
                        cache['Z'].append(pre_activation)
                        
                        # Calculating new values for sums after applying provided activation function on them
                        post_activation: np.ndarray = current_activation_function(pre_activation)
                        # Saving result to cache dictionary
                        cache['A'].append(post_activation)

                        # Assigning post activtion values of sums to variable with current sums
                        current_activations = post_activation

                return current_activations, cache
        
        def predict(self, input_matrix: np.ndarray) -> int:
                """
                predict METHOD

                Parameters:
                        input_matrix: Matrix which contains input values for Neural Network
                
                Returns:
                        predicted_output_index: Index of perceptron in output array, which Neural Network chooses for being the most activated one
                """
                # Front propagation algorithm in action
                final_outputs, _ = self.front_propagation(input_matrix, self._activation_functons)
                # Choosing index of most activated value of output matrix
                predicited_output_index: int = np.argmax(final_outputs)

                return predicited_output_index
                
                        
                        
nn: neural_network = neural_network([2, 2, 1], "xd.npz")
activation_functions = [ReLu, ReLu]

input_matrix: np.ndarray = np.array([2, 2])
nn.assign_activation_functions(activation_functions)
inputs, _ = nn.front_propagation(input_matrix, activation_functions)

print("Weight array first: ", nn.weights_matrixes[0])
print("Weight array second: ", nn.weights_matrixes[1])
print("First input: ", inputs[0][0])



Weight array first:  [[0.85973643 0.42974024]
 [0.59662006 0.82326587]]
Weight array second:  [[0.71232259]
 [0.80551627]]
First input:  4.093424862095548
